In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
from torch import Tensor
import torch.optim as optim


import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from sklearn.model_selection import train_test_split

import numpy as np
import time
import scipy.io

torch.set_default_dtype(torch.float)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name()) 

cpu


In [5]:
# Lucas/this_script.py
import sys
import os
current_dir = os.getcwd()
# Add the 'functions' folder to the Python path
functions_dir = os.path.join(current_dir,'..', 'functions')
sys.path.append(functions_dir)

# Now you can import the function
from Load_data import custom_csv_parser

# Use the function
data = custom_csv_parser('../Patient.csv', True)
#print(data.keys())
print(data.keys())
pde_keys = ['D1', 'D2', 'I_sc','I_p', 'I_eff', 'G', 'G_sc']
patient_keys = ['tau1', 'tau2', 'Ci', 'p2', 'GEZI', 'EGP0', 'Vg', 'taum', 'tausc']
patient_keys_si = ['tau1', 'tau2', 'Ci', 'p2', 'Si', 'GEZI', 'EGP0', 'Vg', 'taum', 'tausc']


Training data loaded successfully!
dict_keys(['Meal_size', 'Steady_insulin', 'Bolus', 'D1', 'D2', 'I_sc', 'I_p', 'I_eff', 'G', 'G_sc', 'tau1', 'tau2', 'Ci', 'p2', 'Si', 'GEZI', 'EGP0', 'Vg', 'taum', 'tausc'])


In [ ]:
Si0 = 0.1

In [ ]:
class DNN(nn.Module):
    '''Network'''
    def __init__(self, layers):
        super(DNN, self).__init__()

        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1] ) for i in range(len(layers)-1)])

        self.activation = nn.Tanh()

        'Initialization'
        for i in range(len(layers) - 1):
            nn.init.xavier_normal_(self.linears[i].weight, gain=1.0)
            nn.init.zeros_(self.linears[i].bias)

    def forward(self, x):
        return None

In [ ]:
def pdes(u, nn_dt, data, Si):
    D1 = u[0]
    D2 = u[1]
    I_sc = u[2]
    I_p = u[3]
    I_eff = u[4]
    G = u[5]
    G_sc = u[6]
    pdes = np.zeros(len(u))
    if t == 0:
        d = data['Meal_size']
        u = data['Bolus']
    else:
        d = 0
        u = data['Steady_insulin']
    pdes[0] = nn_dt[0] + D1 / data['taum'] - d
    pdes[1] = nn_dt[1] - (D1 - D2) / data['taum'] 
    pdes[2] = nn_dt[2] + I_sc / data['tau1'] - u / (data['tau1'] * data['Ci'])
    pdes[3] = nn_dt[3] - (I_sc - I_p) / data['tau2']
    pdes[4] = nn_dt[4] + data['p2'] * I_eff - data['p2'] * Si * I_p
    pdes[5] = nn_dt[5] + (data['GEZI'] + I_eff) * G - data['EGP0'] - 1000 * D2 / (data['Vg'] * data['taum'])
    pdes[6] = nn_dt[6] - (G - G_sc) / data['tausc']

    return pdes

class PINN():
    def __init__(self, layers):
        self.iter = 0

        self.Si = torch.tensor([Si0], requires_grad=True, device=device)
        self.Si = nn.Parameter(self.Si)
        self.nn = DNN(layers).to(device)
        self.nn.register_parameter('Si', self.Si)

        def data_loss(self, t, data):
            u = self.nn(t)  # Outputs 7 x 1 array
            loss = 0
            for i,key in enumerate(data.keys()):
                loss += torch.mean((u[i] - data[key])**2)
            return loss

        def pde_loss(self, t):
            
            t = t.clone()
            t = t.requires_grad_(True)
            u = self.nn(t)
            
            nn_dt = np.zeros(len(u))

            for i in range(len(u)):
                nn_dt[i] = autograd.grad(u[i], t, grad_outputs=torch.ones_like(u[i]), create_graph=True)[0]
            pdes = pdes(u, nn_dt, data, self.Si)
            loss = nn.MSELoss(pdes, torch.zeros_like(pdes))
            return loss
        
        def loss(self, t, data):
            return data_loss(self,t, data) + pde_loss(self, t)
        
        def closure(self):
            optimizer.zero_grad()
            loss = self.loss(t, data)
            loss.backward()
            self.iter += 1

            if self.iter % 100 == 0:
                print("Current loss", loss)
                print(self.Si.item())

            

